This web crawler will scrape the [Napa County Health Department Website](http://ca.healthinspections.us/napa/search.cfm?start=1&1=1&sd=01/01/1970&ed=03/01/2017&kw1=&kw2=&kw3=&rel1=N.permitName&rel2=N.permitName&rel3=N.permitName&zc=&dtRng=YES&pre=similar)

* For each inspection for each facility on a single page, the web crawler will scrape the following information:
    1. Facility name
    2. Address (just street info, not city, state, or zip)
    3. City
    4. State
    5. Zipcode
    6. Inspection date
    7. Inspection typ
    8. For each out-of-compliance violation type, it will scrape the violation type number and corresponding description. For example, an inspection might contain violation type numbers 6 and 7, and descriptions "Adequate handwashing facilities supplied & accessible" and "Proper hot and cold holding temperatures"


* This scraped information will be organized in a SQLite database.




In [1]:
import requests, html, sqlite3
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from IPython.display import display

In [3]:
page_url = (
    "http://ca.healthinspections.us/napa/search.cfm?start=1&1=1&sd=01/01/1970&ed=03/01/2017&kw1=&kw2=&kw3="
    "&rel1=N.permitName&rel2=N.permitName&rel3=N.permitName&zc=&dtRng=YES&pre=similar"
)

def get_connection_obj():
    return sqlite3.connect("ca_inspections_data.db")

def setup_db():
    conn = get_connection_obj()
    with conn:
        c = conn.cursor()
        c.execute("CREATE TABLE IF NOT EXISTS facility (facility_number INTEGER PRIMARY KEY, name TEXT, address TEXT,"
                  "city TEXT, state TEXT, zip TEXT )")
        c.execute("CREATE TABLE IF NOT EXISTS violation_details (violation_number INTEGER PRIMARY KEY, description TEXT)")
        c.execute("CREATE TABLE IF NOT EXISTS inspection_details (facility_number INTEGER REFERENCES facility(facility_number),"
                  "violation_number INTEGER REFERENCES violation_details(violation_number), inspection_date TEXT,"
                  "inspection_type TEXT)")
        conn.commit()
        c.close()

def data_entry(data_frame):
    conn = get_connection_obj()
    with conn:    
        c = conn.cursor()
        invalid_violation_number = -1
        for index, record in data_frame.iterrows():
            c.execute("SELECT * FROM facility WHERE facility_number=?", (record['facility_number'],))
            number_of_records = len(c.fetchall())
            if(number_of_records == 0):
                c.execute("INSERT INTO facility VALUES (?, ?, ?, ?, ?, ?)",
                          (record['facility_number'], record['facility_name'], record['address'],
                           record['city'], record['state'], record['zip_code']))
            
            if(len(record['violation_numbers']) > 0):    
                for index, violationNumber in enumerate(record['violation_numbers']):
                    c.execute("SELECT * FROM violation_details WHERE violation_number=?", (violationNumber,))
                    number_of_records = len(c.fetchall())
                    if(number_of_records == 0):
                        c.execute("INSERT INTO violation_details VALUES (?, ?)",
                                  (violationNumber, record['violation_descriptions'][index]))
                    c.execute("INSERT INTO inspection_details VALUES (?, ?, ?, ?)", 
                          (record['facility_number'], violationNumber,
                           record['inspection_date'], record['inspection_type']))
            else:
                c.execute("INSERT INTO inspection_details VALUES (?, ?, ?, ?)", 
                          (record['facility_number'], invalid_violation_number,
                           record['inspection_date'], record['inspection_type']))
            conn.commit()
            
        c.close()

def display_data():
    conn = get_connection_obj()
    with conn:    
        c = conn.cursor()
        c.execute("SELECT id.facility_number, f.name, f.address, f.city, f.state, f.zip, "
                  "id.violation_number, vd.description, id.inspection_date, id.inspection_type "
                  "FROM inspection_details id "
                  "INNER JOIN facility f "
                  "ON id.facility_number = f.facility_number "
                  "LEFT JOIN violation_details vd "
                  "ON id.violation_number = vd.violation_number")
        records = c.fetchall()
        inspection_df = pd.DataFrame(records, columns=['facility_number', 'facility_name', 'address',
                                                       'city', 'state', 'zip_code', 'violation_number', 
                                                       'violation_description', 'inspection_date', 'inspection_type'])
        display(inspection_df)
        c.close()


def scrape():
    home_page = requests.get(page_url)
    home_page_soup = BeautifulSoup(home_page.text, 'html.parser')
    facilities = home_page_soup.find_all('div', attrs={'style': 'padding:2px;border-top:1px solid #EFEFEF;'})
    base_url = 'http://ca.healthinspections.us'
    inspection_list = []
    
    for facility in facilities:
        address_div = facility.find('div', attrs= {'style': 'margin-bottom:10px;'})
        address = html.unescape(address_div.contents[0]).strip()
        city_state_zip = address_div.text.splitlines()[2].strip().split(',')
        city = city_state_zip[0]
        state = city_state_zip[1].split()[0]
        zip_code = city_state_zip[1].split()[1]
        facility.find_all('a')[-1]['href']
        inspection_report_page = requests.get(urljoin(base_url, facility.find_all('a')[-1]['href']))
        inspection_page_soup = BeautifulSoup(inspection_report_page.text, 'html.parser')
        top_section = inspection_page_soup.find_all('div', attrs={'class': 'topSection'})
        facility_details = top_section[0].find_all('span')
        facility_name = facility_details[0].text
        facility_number = int(facility_details[1].text)
        inspection_date = facility_details[2].text
        inspection_type = facility_details[-1].text
        inspection_tables = inspection_page_soup.find_all('table', attrs={'class': 'insideTable'})
        for inspection_table in inspection_tables:
            rows = inspection_table.findChildren('tr')
            violation_numbers = []
            violation_descriptions = []
            for row in rows:
                if(row.has_attr('class') != True):
                    cells = row.findChildren('td')
                    # Getting the image source for out of compliance violations
                    imageSource = cells[2].find('img')['src']
            
                    # Conditional check to check whether out of compliance is checked 
                    # for this inspection type
                    if 'box_checked_10x10' in imageSource:
                        violationDetails = cells[0].text.split('.')
                        violation_numbers.append(int(violationDetails[0]))
                        violation_descriptions.append(violationDetails[1])
            
        inspection_list.append((facility_number, facility_name, address, city, state, zip_code, inspection_date, inspection_type,
                               violation_numbers, violation_descriptions))
    data_frame = pd.DataFrame(inspection_list, columns=['facility_number', 'facility_name', 'address', 'city', 'state', 'zip_code',
                                                'inspection_date', 'inspection_type', 'violation_numbers', 'violation_descriptions'])

    return data_frame

def main():
    data_frame = scrape()
    setup_db()
    data_entry(data_frame)
    display_data()


if __name__ == '__main__':
    main()

,facility_number,facility_name,address,city,state,zip_code,violation_number,violation_description,inspection_date,inspection_type
0,5823,1313 MAIN STREET,1313 MAIN ST,NAPA,CA,94559,35,Equipment/Utensils - approved; installed; cle...,11/02/2016,Routine
1,5823,1313 MAIN STREET,1313 MAIN ST,NAPA,CA,94559,44,Premises; personal/cleaning items; vermin-pro...,11/02/2016,Routine
2,5823,1313 MAIN STREET,1313 MAIN ST,NAPA,CA,94559,48,Plan Review,11/02/2016,Routine
3,3433,641 MAIN STREET,641 MAIN ST,SAINT HELENA,CA,94574,35,Equipment/Utensils - approved; installed; cle...,11/22/2016,Routine
4,4390,A & A KITCHENS LLC,391 La Fata ST,St Helena,CA,94574,35,Equipment/Utensils - approved; installed; cle...,04/03/2012,Routine
5,5184,A & B MARKET,1855 OLD SONOMA RD,Napa,CA,94558,-1,None,03/10/2016,Routine
6,5178,A DOZEN VINTNERS,3000 St. Helena Hwy N A,SAINT HELENA,CA,94574,35,Equipment/Utensils - approved; installed; cle...,12/02/2011,Routine
7,5178,A DOZEN VINTNERS,3000 St. Helena Hwy N A,SAINT HELENA,CA,94574,41,Plumbing; proper backflow devices,12/02/2011,Routine
8,5178,A DOZEN VINTNERS,3000 St. Helena Hwy N A,SAINT HELENA,CA,94574,41,Plumbing; proper backflow devices,10/03/2012,Routine
9,5176,A&W;/KFC,501 MAIN ST,SAINT HELENA,CA,94574,41,Plumbing; proper backflow devices,02/04/2013,Routine


The above table is generated by joining 3 tables.

1. Facility ( facility_number (Primary Key), name, address, city, state, zip )
2. Violation_Details ( violation_number (Primary Key), description )
3. Inspection_Details ( rowId ( Surrogate Key ), facility_number, violation_number, inspection_date, inspection_type ).
    In case of Inspection_Details, the ideal scenario would be to have a composite primary key consisting of facility_number (Foreign Key), violation_number (Foreign Key), and inspection_date. I have currently not kept the composite key as it would not allow you to run the code
    multiple times.

The data in each of the tables mentioned above is given below:


In [4]:
conn = get_connection_obj()
with conn:    
    c = conn.cursor()
    c.execute("SELECT * FROM facility")
    records = c.fetchall()
    df = pd.DataFrame(records, columns=['facility_number', 'facility_name', 'address',
                                                       'city', 'state', 'zip_code'])
    c.close()
    display(df)

,facility_number,facility_name,address,city,state,zip_code
0,3433,641 MAIN STREET,641 MAIN ST,SAINT HELENA,CA,94574
1,4390,A & A KITCHENS LLC,391 La Fata ST,St Helena,CA,94574
2,5164,ABC FOODS,1825 Old Sonoma RD,Napa,CA,94558
3,5169,A-1 SANDWICH CO (DIET TO GO),429 Cabot RD,S Francisco,CA,94080
4,5172,A-1 FOOD & LIQUOR INC,75 Coombs ST,Napa,CA,94559
5,5176,A&W;/KFC,501 MAIN ST,SAINT HELENA,CA,94574
6,5178,A DOZEN VINTNERS,3000 St. Helena Hwy N A,SAINT HELENA,CA,94574
7,5184,A & B MARKET,1855 OLD SONOMA RD,Napa,CA,94558
8,5823,1313 MAIN STREET,1313 MAIN ST,NAPA,CA,94559


In [6]:
conn = get_connection_obj()
with conn:    
    c = conn.cursor()
    c.execute("SELECT * FROM violation_details")
    records = c.fetchall()
    df = pd.DataFrame(records, columns=['violation_number', 'description'])
    c.close()
    display(df)

,violation_number,description
0,35,Equipment/Utensils - approved; installed; cle...
1,39,Thermometers provided and accurate
2,41,Plumbing; proper backflow devices
3,42,Garbage and refuse properly disposed; facilit...
4,44,Premises; personal/cleaning items; vermin-pro...
5,45,"Floors, walls and ceilings: built, maintained..."
6,48,Plan Review


In [7]:
conn = get_connection_obj()
with conn:    
    c = conn.cursor()
    c.execute("SELECT * FROM inspection_details")
    records = c.fetchall()
    df = pd.DataFrame(records, columns=['facility_number', 'violation_number', 'inspection_date', 'inspection_type'])
    c.close()
    display(df)

,facility_number,violation_number,inspection_date,inspection_type
0,5823,35,11/02/2016,Routine
1,5823,44,11/02/2016,Routine
2,5823,48,11/02/2016,Routine
3,3433,35,11/22/2016,Routine
4,4390,35,04/03/2012,Routine
5,5184,-1,03/10/2016,Routine
6,5178,35,12/02/2011,Routine
7,5178,41,12/02/2011,Routine
8,5178,41,10/03/2012,Routine
9,5176,41,02/04/2013,Routine
